# Feature_Engineering

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("train.csv")

# property_type

'property_type'  sutununu 5 gruba ayirmis

In [ ]:
dict1 = {'Apartment':['Condominium','Timeshare','Loft','Serviced apartment','Guest suite'],
         'House':['Vacation home','Villa','Townhouse','In-law','Casa particular'],
         'Hotel1':['Dorm','Hostel','Guesthouse'],
         'Hotel2':['Boutique hotel','Bed & Breakfast'],
         'Other':['Island','Castle','Yurt','Hut','Chalet','Treehouse',
                  'Earth House','Tipi','Cave','Train','Parking Space','Lighthouse',
                 'Tent','Boat','Cabin','Camper/RV','Bungalow']
        }
dict2 = {i : k for k, v in dict1.items() for i in v}
data['property_type'].replace(dict2).value_counts()

baska bir yol

In [ ]:
# Replacing categories that are types of houses or apartments
df.property_type.replace({
    'Townhouse': 'House',
    'Serviced apartment': 'Apartment',
    'Loft': 'Apartment',
    'Bungalow': 'House',
    'Cottage': 'House',
    'Villa': 'House',
    'Tiny house': 'House',
    'Earth house': 'House',
    'Chalet': 'House'  
    }, inplace=True)

# Replacing other categories with 'other'
df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'Other'

# neighbourhood

mahalle sayisini 619 bulmus,city ye göre gruplamis ve mahalleyi drop etme karari almis

city  neighbourhood	
Boston	34
Chicago	80
DC	    116
LA	    157
NYC	   201
SF	   55

In [ ]:
data['neighbourhood'].nunique()

pd.DataFrame(data.groupby(['city'])['neighbourhood'].nunique())

one hat encoding yapacagi sutunlari bu sekilde secmis

In [ ]:
#columns for get_dummies (one hot encoding)
col_to_getdummies=['property_type','room_type','bed_type','cancellation_policy','city']

#columns that won't be changed
col_no_change=['accommodates','number_of_reviews']

# room_type

room type 3 ,2,1  diye numaralandirmis!

In [ ]:
cleaned_data.room_type = cleaned_data.room_type.apply(lambda x: 3 if x=='Entire home/apt' else 2 if x=='Private room' else 1)

# bed_type

bed_type 2,1 olarak numaralandirmis!

Real Bed         78656
Pull-out Sofa      448
Futon              195
Couch               74
Airbed              50

Most listings have the same bed type so this feature can be dropped

verinin cogu ayni old icin drop etmis 

In [ ]:
df.bed_type.value_counts()
df.drop('bed_type', axis=1, inplace=True)

In [ ]:
cleaned_data.bed_type = cleaned_data.bed_type.apply(lambda x: 2 if x=='Real Bed' else 1)

# cancellation_policy 

cancellation_policy  3,2,1 olarak numaralandirmis

In [ ]:
cleaned_data.cancellation_policy = cleaned_data.cancellation_policy.apply(lambda x: 3 if x=='super_strict_60' else 2 if x=='super_strict_30' else 1)

future larin etkisini görmek icin...

In [ ]:
plot_importance(model)

# host_since

ilk kayit oldugu tarihten gunumuze geen zamana göre kiyaslama yapabilmeyi bu sekilde yapmis

In [ ]:
# Converting to datetime
df.host_since = pd.to_datetime(df.host_since) 

# Calculating the number of days
df['host_days_active'] = (datetime(2019, 4, 9) - df.host_since).astype('timedelta64[D]')

# Printing mean and median
print("Mean days as host:", round(df['host_days_active'].mean(),0))
print("Median days as host:", df['host_days_active'].median())

# Replacing null values with the median
df.host_days_active.fillna(df.host_days_active.median(), inplace=True)

# host_response_rate

Bu işlemlerle df.host_response_rate sütunu kategorilere ayrılmış, eksik değerler 'unknown' olarak işaretlenmiş ve her bir kategoriye ait değerlerin sayıları hesaplanmıştır.

100%       37730
unknown    25699
50-89%      7346
90-99%      6519
0-49%       2377

bu sonuc alinmis

In [ ]:
# Bin into four categories
df.host_response_rate = pd.cut(df.host_response_rate, bins=[0, 50, 90, 99, 100], labels=['0-49%', '50-89%', '90-99%', '100%'], include_lowest=True)

# Converting to string
df.host_response_rate = df.host_response_rate.astype('str')

# Replace nulls with 'unknown'
df.host_response_rate.replace('nan', 'unknown', inplace=True)

# Category counts
df.host_response_rate.value_counts()

# amenities

'amenities' sütunu, dahil edilen hizmetlerin bir listesini içerir; yararlı olabilir, bu yüzden onu ayrıştıralım:

In [ ]:
amenities = {x for xs in df_pd['amenities'].tolist() for x in xs}
amenities.remove('')
amenities.remove('translation missing: en.hosting_amenity_49')
amenities.remove('translation missing: en.hosting_amenity_50')
amenities = list(amenities)


In [ ]:
amenities_df = pd.DataFrame(df_pd['amenities'].apply(lambda x: [amenity in x for amenity in amenities]).tolist(), columns=amenities, index=df_pd.index)
amenities_df

amenities sütununun bar plot olarak gösterilmesi

In [ ]:
amenities_df.astype('float').sum().sort_values(ascending=False)[:30].plot.bar()

amenities sutunundaki degerler

In [ ]:
# Creating a set of all possible amenities
amenities_list = list(df.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.split(',')]
amenities_set = set(amenities_set)
amenities_set

In [ ]:
df.loc[df['amenities'].str.contains('24-hour check-in'), 'check_in_24h'] = 1
df.loc[df['amenities'].str.contains('Air conditioning|Central air conditioning'), 'air_conditioning'] = 1
df.loc[df['amenities'].str.contains('Amazon Echo|Apple TV|Game console|Netflix|Projector and screen|Smart TV'), 'high_end_electronics'] = 1
df.loc[df['amenities'].str.contains('BBQ grill|Fire pit|Propane barbeque'), 'bbq'] = 1
df.loc[df['amenities'].str.contains('Balcony|Patio'), 'balcony'] = 1
df.loc[df['amenities'].str.contains('Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'), 'nature_and_views'] = 1
df.loc[df['amenities'].str.contains('Bed linens'), 'bed_linen'] = 1
df.loc[df['amenities'].str.contains('Breakfast'), 'breakfast'] = 1
df.loc[df['amenities'].str.contains('TV'), 'tv'] = 1
df.loc[df['amenities'].str.contains('Coffee maker|Espresso machine'), 'coffee_machine'] = 1
df.loc[df['amenities'].str.contains('Cooking basics'), 'cooking_basics'] = 1
df.loc[df['amenities'].str.contains('Dishwasher|Dryer|Washer'), 'white_goods'] = 1
df.loc[df['amenities'].str.contains('Elevator'), 'elevator'] = 1
df.loc[df['amenities'].str.contains('Exercise equipment|Gym|gym'), 'gym'] = 1
df.loc[df['amenities'].str.contains('Family/kid friendly|Children|children'), 'child_friendly'] = 1
df.loc[df['amenities'].str.contains('parking'), 'parking'] = 1
df.loc[df['amenities'].str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'outdoor_space'] = 1
df.loc[df['amenities'].str.contains('Host greets you'), 'host_greeting'] = 1
df.loc[df['amenities'].str.contains('Hot tub|Jetted tub|hot tub|Sauna|Pool|pool'), 'hot_tub_sauna_or_pool'] = 1
df.loc[df['amenities'].str.contains('Internet|Pocket wifi|Wifi'), 'internet'] = 1
df.loc[df['amenities'].str.contains('Long term stays allowed'), 'long_term_stays'] = 1
df.loc[df['amenities'].str.contains('Pets|pet|Cat(s)|Dog(s)'), 'pets_allowed'] = 1
df.loc[df['amenities'].str.contains('Private entrance'), 'private_entrance'] = 1
df.loc[df['amenities'].str.contains('Safe|Security system'), 'secure'] = 1
df.loc[df['amenities'].str.contains('Self check-in'), 'self_check_in'] = 1
df.loc[df['amenities'].str.contains('Smoking allowed'), 'smoking_allowed'] = 1
df.loc[df['amenities'].str.contains('Step-free access|Wheelchair|Accessible'), 'accessible'] = 1
df.loc[df['amenities'].str.contains('Suitable for events'), 'event_suitable'] = 1

#https://github.com/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb

In [ ]:
# Replacing nulls with zeros for new columns
cols_to_replace_nulls = df.iloc[:,41:].columns
df[cols_to_replace_nulls] = df[cols_to_replace_nulls].fillna(0)

# Produces a list of amenity features where one category (true or false) contains fewer than 10% of listings
infrequent_amenities = []
for col in df.iloc[:,41:].columns:
    if df[col].sum() < len(df)/10:
        infrequent_amenities.append(col)
print(infrequent_amenities)

# Dropping infrequent amenity features
df.drop(infrequent_amenities, axis=1, inplace=True)

# Dropping the original amenity feature
df.drop('amenities', axis=1, inplace=True)

df.columns[40:]



#https://github.com/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb

About a quarter of listings have not had a review written for them. This is too large a proportion of the dataset to drop, and dropping the columns would lose a lot of useful information - reviews are very important in people's decisions to book, and therefore price.

This is also too large a proportion of the dataset to simply replace with median/mean values, as this would skew the distribution substantially. Also, the missing values here are not really missing values, as the fact that they are NaNs is meaningful - it tells us that these are new or previously unbooked listings that have not had reviews yet. In order to make the resulting model work able to predict prices for any Airbnb listing, including brand new listings, is is actually beneficial to keep them in. Therefore, these will be kept as an 'unknown' category, and the feature will have to be treated as categorical (and therefore one-hot encoded) rather than numerical.

Bu aynı zamanda, dağılımı büyük ölçüde çarpıtacağından, basitçe ortanca/ortalama değerlerle değiştirilemeyecek kadar büyük bir veri kümesi oranıdır. Ayrıca, buradaki eksik değerler gerçekten eksik değerler değildir, çünkü NaN'ler olmaları anlamlıdır - bu bize bunların yeni veya önceden rezerve edilmemiş ve henüz inceleme yapılmamış listeler olduğunu söyler. Ortaya çıkan modelin, yepyeni kayıtlar da dahil olmak üzere herhangi bir Airbnb kaydının fiyatlarını tahmin edebilecek şekilde çalışmasını sağlamak için, onları içeride tutmak aslında faydalıdır. Bu nedenle, bunlar 'bilinmeyen' bir kategori olarak tutulacak ve özellik, sayısaldan ziyade kategorik (ve bu nedenle bir sıcak kodlanmış) olarak ele alınmalıdır.

In [ ]:
print(f"Null values in 'first_review': {round(100*df.first_review.isna().sum()/len(df),1)}%")
print(f"Null values in 'review_scores_rating': {round(100*df.review_scores_rating .isna().sum()/len(df),1)}%")

In [ ]:
df.first_review = pd.to_datetime(df.first_review) # Converting to datetime

# Calculating the number of days between the first review and the date the data was scraped
df['time_since_first_review'] = (datetime(2019, 4, 9) - df.first_review).astype('timedelta64[D]')

In [ ]:
# Distribution of the number of days since first review
df.time_since_first_review.hist(figsize=(15,5), bins=30);

In [ ]:
# Plotting feature importances
plt.figure(figsize=(8,20))
plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()